Import der Pakete


In [1]:
import pandas as pd
import openpyxl

In [ ]:
file_path_required_data = "../../data/required_info.csv"
file_path_ids = "../../data/spiele_ids_steam.csv"


In [ ]:
required_data = pd.read_csv(file_path_required_data)

required_data.to_excel("../../data/required_info.xlsx", sheet_name="Sheet1", index=False, engine="openpyxl") # Nur um etwas herauszufinden

C:\Users\Flo\AppData\Local\Temp\ipykernel_27708\844105570.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  required_data = pd.read_csv(file_path_required_data)


C:\Users\Flo\AppData\Local\Temp\ipykernel_20032\314828543.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  required_data = pd.read_csv(file_path_required_data) 

Bedeutet für mich, dass ich am Anfang meiner Datenbeschaffung schlampig gearbeitet habe oder aber die Daten bei Steam fehler haben. Das werden wir nun herausfinden

In [ ]:
column_1 = required_data.iloc[:,1] # Alter spalte. Sollte eigentlich nur Ints haben schauen wir uns mal den print an um sowohl start Werte als auch Endwerte zu sehen

In [ ]:
print(column_1)

0           16
1           12
2           16
3            6
4           16
          ... 
124421     6.0
124422     6.0
124423    14.0
124424    16.0
124425    16.0
Name: alter, Length: 124426, dtype: object


Wie wir sehen, sind sowhl ints als auch floats in column 1 -> Alter enthalten. Das ganze fixen wir nun indem wir alle zu Integers machen.

In [ ]:
column_1.unique()

array(['16', '12', '6', '0', '18', nan, 't', 'm', '3', '7', 'e', 'e10',
       '15', '14', 'rp', '10', '11', '2147483647', '4', 12.0, 0.0, 6.0,
       16.0, 18.0, 14.0, 2147483647.0, 10.0], dtype=object)

Wie wir sehen können ist die Formatierung nicht einheitlich bei Steam und weist deutliche Fehler auf. Kann auch sein das mein Script einfach Fehlerhaft ist und nicht robust genug für alle Fälle. Wie folgender Wert: 2147483647 oder e10 oder e. Ich kann mir jetzt natürlich die Mühe machen und diese Werte überprüfen und nochmal die API überprüfen aber ich gehe mal davon aus, dass es wenige dieser Werte gibt. Das ganze überprüfe ich jetzt auch nochmal

In [ ]:
column_1.value_counts()

alter
0               19621
0.0             16248
12              14286
12.0            13793
16              11976
16.0            11510
6               10499
6.0              8701
18               3130
18.0             2924
3                 749
7                 409
14                216
14.0              155
11                 91
2147483647         61
2147483647.0       42
15                 41
10                 25
4                  18
10.0               18
m                  11
e                   6
t                   6
e10                 3
rp                  3
Name: count, dtype: int64

Es handel sich hier also um etwa 150 Werte. Die Zeilen mit den IDs ziehe ich mir also nun und überprüfe das ganze nochmal in der API. Schließlich könnte dahinter ja das nächste Spiel stecken. Wäre es nur ein einzelner Wert gewesen, wäre es mir egal und zu Zeitaufwendig. Wenn ich merke, dass ich länger als 30 Minuten hierfür brauchen sollte, weil jedes der Spiele unterschiedliche Keywords benutzt bin ich an der Stelle auch raus.


In [ ]:
required_data[required_data.isna().any(axis=1)]

,id,alter,genre,categories,released,eigenschaften,original_price,recommendations
5,60,0,Action,Multi-player/ PvP/ Online PvP/ Custom Volume C...,"1 Nov, 2000",NaN,499.0,4387.0
15,360,16,Action,Multi-player/ Camera Comfort/ Custom Volume Co...,"1 May, 2006",Drastische Gewalt,NaN,4037.0
24,630,16,Action/ Free To Play,Single-player/ Multi-player/ Co-op/ Steam Achi...,"19 Jul, 2010",Drastische Gewalt,NaN,299.0
32,1510,6,Indie/ Strategy,Single-player/ Family Sharing,"23 Aug, 2006",NaN,1179.0,1717.0
33,1520,0,Indie/ Strategy,Single-player/ Multi-player/ PvP/ Online PvP/ ...,"29 Sep, 2006",NaN,1179.0,3369.0
...,...,...,...,...,...,...,...,...
124421,4072460,6.0,Action; Casual; Indie,Single-player; Custom Volume Controls; Mouse O...,October 2025,Fantasy-Gewalt,NaN,NaN
124422,4072650,6.0,Adventure; Indie,Single-player; Steam Achievements; Camera Comf...,To be announced,NaN,NaN,NaN
124423,4073000,14.0,Adventure; Indie; Simulation,Single-player; Steam Achievements; Full contro...,To be announced,Drogen; Schimpfwörte,NaN,NaN
124424,4073900,16.0,Casual; Indie; RPG; Simulation,Single-player; Full controller support; Camera...,Coming soon,Horror,NaN,NaN


Mein logischer Schritt als nächstes ist, die Spieleliste zu nehmen und dann zu überprüfen ob die Spiele in Deutschland noch verfügbar sind. Das kann ich glücklicherweise ganz einfach machen indem ich einfach wieder die Shopseite ansteuer. Kommt also dann ein Fehler, droppe ich direkt das Spiel. Warum sollte ich nämlich ein Spiel empfohlen bekommen, was ich gar nicht spielen kann. 

Folgendes habe ich bei meiner Recherche gefunden: https://steamcommunity.com/discussions/forum/24/4843149028231900131/.
Spieleentwickler / Developer etc. haben anscheinend versagt und deswegen kann ich wohl nicht auf ganz viele spiele zugreifen. Ich werde nun die Nas exportieren und nochmal gegenprüfen. In meinem Code ist nämlich ein Dicker Fehler

In [ ]:
df_na = required_data[required_data.isna().any(axis=1)]
df_na.to_csv("../../data/missing_data_facecheck.csv", index=False)


In [ ]:
Liste_drop_id = ["rp","e10","e","t","m"]

In [ ]:
required_data = required_data.drop(
    required_data[required_data["alter"].isin(Liste_drop_id)].index
)

required_data["alter"] = pd.to_numeric(required_data["alter"])

required_data = required_data.drop(
    required_data[required_data["alter"]== 2147483647].index
)
required_data = required_data.drop(
    required_data[required_data["alter"]== 2147483647.0].index
)


required_data = required_data.drop(required_data[required_data["alter"].isna()].index)

In [ ]:
spiele_namen = pd.read_csv(file_path_ids)

In [ ]:
merged = pd.merge(required_data, spiele_namen, how="left", left_on="id", right_on="appid")

In [ ]:
full_data = merged.drop(columns=["appid"])

In [ ]:
full_data

,id,alter,genre,categories,released,eigenschaften,original_price,recommendations,name
0,10,16.0,Action,Multi-player/ PvP/ Online PvP/ Shared/Split Sc...,"1 Nov, 2000",Drastische Gewalt,819.0,162400.0,Counter-Strike
1,20,12.0,Action,Multi-player/ PvP/ Online PvP/ Shared/Split Sc...,"1 Apr, 1999",Gewalt,499.0,6661.0,Team Fortress Classic
2,30,16.0,Action,Multi-player/ Camera Comfort/ Color Alternativ...,"1 May, 2003",Drastische Gewalt,499.0,4319.0,Day of Defeat
3,40,6.0,Action,Multi-player/ PvP/ Online PvP/ Shared/Split Sc...,"1 Jun, 2001",Fantasy-Gewalt,499.0,2341.0,Deathmatch Classic
4,50,16.0,Action,Single-player/ Multi-player/ Custom Volume Con...,"1 Nov, 1999",Drastische Gewalt,499.0,22943.0,Half-Life: Opposing Force
...,...,...,...,...,...,...,...,...,...
114405,4072460,6.0,Action; Casual; Indie,Single-player; Custom Volume Controls; Mouse O...,October 2025,Fantasy-Gewalt,NaN,NaN,Crownguard
114406,4072650,6.0,Adventure; Indie,Single-player; Steam Achievements; Camera Comf...,To be announced,NaN,NaN,NaN,What Was Found at Ravenhill
114407,4073000,14.0,Adventure; Indie; Simulation,Single-player; Steam Achievements; Full contro...,To be announced,Drogen; Schimpfwörte,NaN,NaN,Shake It: Lycoris
114408,4073900,16.0,Casual; Indie; RPG; Simulation,Single-player; Full controller support; Camera...,Coming soon,Horror,NaN,NaN,The Lightning Girl


Nun erstellen wir ein Drop für den fall, dass trotz des "game" filters in steam, Demos oder Dlcs enthalten sind

In [ ]:
len(full_data)

114410

In [ ]:
# Filter nach Witespace, Bindestrich oder Unterstrich
full_data_v1 = full_data.copy()
full_data_v1 = full_data_v1.drop(full_data_v1[full_data_v1["name"].str.contains(r"[\s\-_]Demo[\s\-_]?", case=False, na=False, regex=True)
].index)




In [ ]:
len(full_data_v1)

114205

In [ ]:
full_data_v1["alter"] = full_data_v1["alter"].astype(int)


In [ ]:
sum(full_data_v1["original_price"].isna())

41507

Die Preise sind jetzt weniger das Problem.
- Im Store kann ich per html auf die Preise auch zugreifen.

In [ ]:
full_data_v1["categories"] = full_data_v1["categories"].str.replace(";", "/").str.replace("/ ", "/")

In [ ]:
full_data_v1.tail()

,id,alter,genre,categories,released,eigenschaften,original_price,recommendations,name
114405,4072460,6,Action; Casual; Indie,Single-player/Custom Volume Controls/Mouse Onl...,October 2025,Fantasy-Gewalt,NaN,NaN,Crownguard
114406,4072650,6,Adventure; Indie,Single-player/Steam Achievements/Camera Comfor...,To be announced,NaN,NaN,NaN,What Was Found at Ravenhill
114407,4073000,14,Adventure; Indie; Simulation,Single-player/Steam Achievements/Full controll...,To be announced,Drogen; Schimpfwörte,NaN,NaN,Shake It: Lycoris
114408,4073900,16,Casual; Indie; RPG; Simulation,Single-player/Full controller support/Camera C...,Coming soon,Horror,NaN,NaN,The Lightning Girl
114409,4076370,16,Casual; Indie,Single-player/Steam Achievements/Full controll...,Coming soon,Horror,NaN,NaN,ICU Anomaly


## Jetzt geht es an die Spalte "released"
- alle Spiele die noch nicht released sind, werden entfernt

In [ ]:
full_data_v1["released"].value_counts()

released
Coming soon        12343
To be announced     7926
2025                2009
2026                1749
Q4 2025             1065
                   ...  
23 Jun, 2026           1
29 Mar, 2026           1
15 Apr, 2026           1
1 Sep, 2027            1
Oct 9, 2025            1
Name: count, Length: 5314, dtype: int64

In [ ]:
import pandas as pd

# Q, Zahl, Leerzeichen(optional), 4 stellige dezimal, Monate, und der andere Quatsch
pattern = r"(Q\d\s?\d{4}|January|February|March|April|May|June|July|August|September|October|November|December|Coming soon|To be announced)"

full_data_v1 = full_data_v1[~full_data_v1["released"].str.contains(pattern, case=False, na=False)]

mask = full_data_v1["released"].astype(str).str.fullmatch(r"\d{4}")

full_data_v1 = full_data_v1[~mask]

full_data_v1 = full_data_v1.drop(full_data_v1[full_data_v1["released"].isna()].index)



C:\Users\Flo\AppData\Local\Temp\ipykernel_27708\3529659427.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  full_data_v1 = full_data_v1[~full_data_v1["released"].str.contains(pattern, case=False, na=False)]


In [ ]:
len(full_data_v1)

79374

In [ ]:
full_data_v1.tail()

,id,alter,genre,categories,released,eigenschaften,original_price,recommendations,name
114381,4066370,0,Adventure; Casual,Single-player/Family Sharing,"3 Oct, 2025",NaN,1379.0,NaN,Fragments of Truth: An MCF Story Collector's E...
114385,4066470,16,Adventure; Indie; Simulation,Single-player/Steam Achievements/Camera Comfor...,"16 Oct, 2025",Horror,NaN,NaN,Just a Stay at the Inn (旅館に泊まっただけなのに)
114388,4067390,12,Action,Single-player/Full controller support/Family S...,"14 Oct, 2025",Gewalt,179.0,NaN,LuminousStory
114389,4067540,12,Action; Indie; Free To Play,Single-player/Camera Comfort,"15 Oct, 2025",Gewalt,NaN,NaN,Vacubot
114400,4069920,18,Action; Adventure; Indie,Single-player/Multi-player/Co-op/LAN Co-op/Cro...,"24 Oct, 2025",Drastische Gewalt; Derbe Sprache,NaN,NaN,Chornobyl Breakout


In [ ]:
full_data_v1["released"] = pd.to_datetime(
    full_data_v1["released"],
    format="mixed",
    errors= "coerce"
)


In [ ]:
full_data_v1 = full_data_v1.drop(full_data_v1[full_data_v1["released"].isna()].index)

In [ ]:
full_data_v1.to_csv("../../data/aufbereitete_daten_v1.csv", index=False) 

Nun werden Listen für die Eigenschaften und die Categories erstellt.

In [ ]:
cat = full_data_v1["categories"] 


Series([], Name: categories, dtype: object)

In [ ]:
cat_list = cat.str.split("/") ## Achtung Das Splittet auch die Kategorie N/A
cat_list = cat_list.fillna("NA")

In [ ]:
# Wir können hier mit einem set arbeiten, weil die Reihenfolge egal ist

In [ ]:
categories_liste = []

for list in cat_list:
    for categs in list:
        if categs not in categories_liste:
            categories_liste.append(categs)

print(categories_liste)

['Multi-player', 'PvP', 'Online PvP', 'Shared', 'Split Screen PvP', 'Color Alternatives', 'Custom Volume Controls', 'Keyboard Only Option', 'Stereo Sound', 'Valve Anti-Cheat enabled', 'Family Sharing', 'Remote Play Together', 'Single-player', 'Adjustable Difficulty', 'Playable without Timed Input', 'Save Anytime', 'Full controller support', 'Steam Cloud', 'Remote Play on Phone', 'Remote Play on Tablet', 'Steam Achievements', 'Steam Trading Cards', 'Captions available', 'Steam Workshop', 'Camera Comfort', 'Subtitle Options', 'Surround Sound', 'Includes Source SDK', 'Commentary available', 'Steam Timeline', 'Cross-Platform Multiplayer', 'Stats', 'Includes level editor', 'In-App Purchases', 'Partial Controller Support', 'Co-op', 'Steam Leaderboards', 'Remote Play on TV', 'Online Co-op', 'SteamVR Collectibles', 'Split Screen Co-op', 'Split Screen', 'Adjustable Text Size', 'LAN PvP', 'LAN Co-op', 'MMO', 'Mods', 'Mods (require HL2)', 'Mouse Only Option', 'N', 'A', 'Solo', 'Multijoueur', 'Coo

In [ ]:
pd.Series(categories_liste).to_csv("../../data/Spielekategorien_unfinished.csv")

In [ ]:
eig = full_data_v1["eigenschaften"]
eig.tail()

114381                                 NaN
114385                              Horror
114388                              Gewalt
114389                              Gewalt
114400    Drastische Gewalt; Derbe Sprache
Name: eigenschaften, dtype: object

In [ ]:
eig_list = eig.fillna("-")
eig_list = eig_list.str.split(";")


In [ ]:
eigenschaften = []

for list in eig_list:
    for eigen in list:
        if eigen not in eigenschaften:
            eigenschaften.append(eigen)

print(eigenschaften)

['Drastische Gewalt', 'Gewalt', 'Fantasy-Gewalt', '-', 'Alkohol / Tabak/ Gewalt', 'Schimpfwörte', 'Drastische Gewalt/ Schimpfwörte', 'Violence/ Bad language', 'Violence', 'Drastische Gewalt/ Sexuelle Andeutungen/ Schimpfwörte', 'Drastische Gewalt/ Alkohol / Tabak/ Derbe Sprache', 'Gewalt/ Schimpfwörte', 'Drastische Gewalt/ Drogen', 'Gewalt/ Alkohol / Tabak', 'Alkohol / Tabak', 'Drastische Gewalt/ Sexuelle Andeutungen', 'Gewalt/ Derbe Sprache', 'Drastische Gewalt/ Derbe Sprache', 'Realistic looking violence - Mild bad language', ' Bad Language', 'Nonrealistic looking violence towards fantasy characters', 'Non realistic looking violence towards fantasy characters - Realistic looking violence', 'Non realistic looking violence towards human characters', ' Derbe Sprache', ' Alkohol / Tabak', '- Extreme violence', ' ', ' - Sexual violence', ' - Strong language', 'Bad Language', 'Realistic looking violence - Strong language', 'Extreme violence - Violence towards defenceless people - Glamorisa

In [ ]:
pd.Series(eigenschaften).to_csv("../../data/Spieleeigenschaften_unfinished.csv")